In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from io import BytesIO
from PIL import Image
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
sio = socketio.Server()
app = Flask(__name__)

In [3]:
path_to_model = r"C:\Users\adm.m.gosciniak\Desktop\workspace\ANN-ML\beta_simulator_windows\mymodel.h5"
model = load_model(path_to_model)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalize (Lambda)           (None, 60, 80, 3)         0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 58, 78, 64)        1792      
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 56, 76, 32)        18464     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 26, 36, 16)        4624      
_______________________________________

In [4]:
def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [ ]:
def process_image(img):
    return img[10:130:2,::4,:]

@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        
        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
        
        img = process_image(image_array)
        
        img_batch = np.expand_dims(img, axis=0)
        
        steering_angle = float(model.predict(img_batch))
        throttle = 0.1         # -1.0 .. 1.0

        if speed < 10:
            throttle = 0.6
        if speed > 15:
            throttle = -0.1
        
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('',4567)), app)

(14600) wsgi starting up on http://0.0.0.0:4567
(14600) accepted ('127.0.0.1', 57671)
127.0.0.1 - - [29/Aug/2019 12:04:00] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 50.204721
(14600) accepted ('127.0.0.1', 57814)
